In [1]:
import pandas as pd
import numpy as np
from io import StringIO

def squeeze_nan_left(x):
            original_columns = x.index.tolist()
            squeezed = x.dropna()
            squeezed.index = [original_columns[n] for n in range(squeezed.count())]
            return squeezed.reindex(original_columns, fill_value=np.nan)

In [6]:
#Everi - https://www.sec.gov/Archives/edgar/data/1318568/000155837017002779/evri-20170523xdef14a.htm
#MNTA - https://www.sec.gov/Archives/edgar/data/1235010/000123501017000006/mnta2017proxystatement.htm

url = 'https://www.sec.gov/Archives/edgar/data/1318568/000155837017002779/evri-20170523xdef14a.htm'

data = pd.read_html(url)


In [62]:
SCT.index.tolist()
SCT2 = [SCT.index.tolist()[n] for n in range(SCT.index.tolist().count())]

TypeError: count() takes exactly one argument (0 given)

In [8]:
#168 for Everi
#128 for MNTA

sctind = 168
SCT = data[sctind]
#SCT
SCT.apply(squeeze_nan_left, axis=1).dropna(axis = 1, how = 'all').dropna(axis = 0, how = 'all')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
1,Name and principal position,Year,Salary,Bonus(1),Stockawards(2),Optionawards(3),Non-equityincentive plancompensation(4),All othercompensation(5),Total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Michael D. Rumbolz,2016,$,507692,(6),$,132377,$,-,$,601162,$,-,$,17348.0,(7),$,1258579.0
4,President and Chief Executive Officer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Randy L. Taylor,2016,400000,65000,-,215959,-,9779,690738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"Executive Vice President, Chief Financial Officer",2015,389423,-,-,930000,-,15568,1.33499e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2014,275962,-,313280,601310,-,11501,1.20205e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,Juliet A. Lim (8),2016,400000,65000,-,215959,-,9779,690738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,"Executive Vice President, Payments Business Le...",2015,397308,-,-,930000,-,15957,1.34326e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,2014,266539,-,341760,601310,-,46164,1.25577e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,David J. Lucchese (9),2016,425000,53125,-,215959,-,9818,703902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
